## Profiles

In [ ]:
import pedpy as pp
from pedpy import (
    get_grid_cells,
    compute_grid_cell_polygon_intersection_area,
)
from pedpy.column_identifier import SPEED_COL, FRAME_COL, ID_COL

import matplotlib.pyplot as plt
import glob
from pathlib import Path
import pickle

In [ ]:
files = sorted(glob.glob("AppData/*.txt"))
area = [[-6, 0], [5, 0], [5, 7], [-6, 7]]
walkable_area = pp.WalkableArea(area)
trajectories = {}
for file in files:
    trajectories[file] = pp.load_trajectory(trajectory_file=Path(file),default_frame_rate=30,
        default_unit=pp.TrajectoryUnit.METER,)

In [ ]:
voronoi_polygons = {}
for idx, (filename, traj) in enumerate(trajectories.items()):
    voronoi_polygons[filename] = pp.compute_individual_voronoi_polygons(
        traj_data=traj, walkable_area=walkable_area
    )


#individual_cutoff = pp.compute_individual_voronoi_polygons(
#    traj_data=traj,
#    walkable_area=walkable_area,
#    cut_off=Cutoff(radius=0.8, quad_segments=3),
#)

In [ ]:
density_voronoi = {}
speed_voronoi = {}
individual_speed = {}
intersecting = {}
for idx, (filename, traj) in enumerate(trajectories.items()):
    density_voronoi[filename], intersecting = pp.compute_voronoi_density(
        individual_voronoi_data=voronoi_polygons[filename],
        measurement_area=walkable_area,
    )
    individual_speed[filename] = pp.compute_individual_speed(
        traj_data=traj,
        frame_step=1,
        compute_velocity=True,
        speed_calculation=pp.SpeedCalculation.BORDER_SINGLE_SIDED,
    )
    speed_voronoi[filename] = pp.compute_voronoi_speed(
        traj_data=traj,
        individual_voronoi_intersection=intersecting,
        measurement_area=walkable_area,
        individual_speed=individual_speed[filename],
    )

In [ ]:
profile_data = {}
for idx, (filename, traj) in enumerate(trajectories.items()):
    print(filename)
    profile_data[filename] = individual_speed[filename].merge(voronoi_polygons[filename], on=[ID_COL, FRAME_COL])
    profile_data[filename] = profile_data[filename].merge(traj.data, on=[ID_COL, FRAME_COL])

In [ ]:
import pickle
profile_file = "AppData/profile_data.pkl"
with open(profile_file, "wb") as f:
    pickle.dump(profile_data, f)

In [ ]:
grid_size = 0.4
grid_cells, _, _ = get_grid_cells(
    walkable_area=walkable_area, grid_size=grid_size
)
(grid_cell_intersection_area,resorted_profile_data,) = compute_grid_cell_polygon_intersection_area(data=profile_data, grid_cells=grid_cells)

In [ ]:
voronoi_density_profile = pp.compute_density_profile(
    data=resorted_profile_data,
    walkable_area=walkable_area,
    grid_intersections_area=grid_cell_intersection_area,
    grid_size=grid_size,
    density_method=DensityMethod.VORONOI,
)

In [ ]:
voronoi_speed_profile = compute_speed_profile(
    data=resorted_profile_data,
    walkable_area=walkable_area,
    grid_intersections_area=grid_cell_intersection_area,
    grid_size=grid_size,
    speed_method=SpeedMethod.VORONOI,
)

In [ ]:
from pedpy import plot_profiles
import matplotlib.pyplot as plt

rmax=4 
vmax=2
jmax=5
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, layout="constrained")
fig.set_size_inches(12, 5)
fig.suptitle("Density profile")
cm = plot_profiles(
    walkable_area=walkable_area,
    profiles=voronoi_density_profile,
    axes=ax0,
    label=r"$\\rho$ / 1/$m^2$",
    vmin=0,
    vmax=rmax,
    #title="Voronoi",
)
#colorbar_ax = fig.axes[]
#colorbar_ax.set_ylabel("$\\rho$ / 1/$m^2$", size=18)
#colorbar_ax.tick_params(labelsize=18)
cm = plot_profiles(
    walkable_area=walkable_area,
    profiles=speed_profiles,
    axes=ax1,
    label=r"$v / m/s$",
    vmin=0,
    vmax=vmax,
    #title="Speed",
)
fig.tight_layout(pad=2)
cm = plot_profiles(
    walkable_area=walkable_area,
    profiles=voronoi_density_profile,speed_profiles
    axes=ax2,
    label=r"$J$ / 1/$m.s$",
    vmin=0,
    vmax=8,
    #title="Classic",
)

for ax in [ax0, ax1, ax2]:
    ax.tick_params(axis="x", length=0)
    ax.tick_params(axis="y", length=0)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels([])
    ax.set_yticklabels([])


plt.show()